In [10]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models.widgets import CheckboxButtonGroup
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import Legend
import xarray as xr
import pandas as pd
from glob import glob
from ipywidgets import interact
from bokeh.io import export_png, export_svgs
import os
from os.path import join

output_notebook()

Loading BokehJS ...

## Plot Ice Concentration

In [13]:
def save_fname(ds, name):
    direc = ds.filename.replace('.nc', '_plts')
    try:
        os.mkdir(direc)
    except IOError:
        pass
    fname = join(direc, name + '.svg')
    print(fname)
    return fname

def line_plot(ds, bias_stddev):
    p = figure(x_axis_type="datetime", plot_width=500, plot_height=int(500/1.618))
    p.output_backend = "svg"
    #p.title.text = 'Click on legend entries to mute the corresponding lines'
    legend_it = []
    ds = ds[[dv for dv in ds.data_vars if ds[dv].dims == ('time',)]]
#     ds = ds.dropna(dim='time')
    for name, color in (('ice', 'crimson'), ('water', 'royalblue')):
        name += '_' + bias_stddev
        x, y = ds.time.values, ds[name].values
        c = p.line(x, y, line_color=color)
        p.circle(x, y, line_color=color, fill_color='white')
        legend_it.append((name.replace('_', ' '), [c]))
#     p.legend.location = "right"
    p.xaxis.axis_label = "Date"
    if bias_stddev == 'bias':
        p.yaxis.axis_label = "Bias (-)"
    elif bias_stddev == 'stddev':
        p.yaxis.axis_label = "Standard Deviation (-)"
    #p.legend.click_policy = "hide"
    legend = Legend(items=legend_it, location=(0, -5), margin=0)
    legend.click_policy="hide"
    p.add_layout(legend, 'above')
    p.legend.orientation = "horizontal"
    return p
    

def bar_plot(ds):
    p = figure(x_axis_type='datetime', plot_width=500, plot_height=int(500/1.618))
    p.output_backend = "svg"
    df = ds[['within_10pct', 'within_20pct']].to_dataframe()
    width = 6 if len(df) < 100 else 0.5
    source = ColumnDataSource(df)
    p.vbar(x='time', source=source,
           bottom=0, top='within_10pct', width=width, line_width=width, color='royalblue', legend='Within 10%')
    p.vbar(x='time', source=source,
           bottom='within_10pct', top='within_20pct', width=width, line_width=width, color='crimson', legend='Within 20%')
    p.legend.location = "bottom_left"
    return p

def make_plots(glob_str, from_date):
    for fn in glob(glob_str):
        print(fn)
        ds = xr.open_mfdataset(fn)
        ds.attrs = {'filename': fn}
        ds = ds.drop('source')

        ds = ds.sel(time=slice(from_date, None))
        p = line_plot(ds, 'bias')
        show(p)
        export_svgs(p, filename=save_fname(ds, 'bias'))
        p = line_plot(ds, 'stddev')
        export_svgs(p, filename=save_fname(ds, 'stddev'))
        show(p)
        p = bar_plot(ds)
        show(p)
        export_svgs(p, filename=save_fname(ds, 'match'))
        
        
make_plots('/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_*.nc', '2018-06-01')
make_plots('/data2/validations/validate_conc_ice_conc_polstere-100_amsr2-tud_*.nc', '2018-06-01')
make_plots('/data2/validations/validate_conc_ice_conc_polstere-100_multi_*H_20000101_20190630.nc', '2001-06-01')
make_plots('/data2/validations/validate_conc_ice_conc_polstere-100_multi_*H_20000101_20190630.nc', '2018-06-01')

/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_SH_20180601_20190601.nc


/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_SH_20180601_20190601_plts/bias.svg
/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_SH_20180601_20190601_plts/stddev.svg


/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_SH_20180601_20190601_plts/match.svg
/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_NH_20180601_20190601.nc


/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_NH_20180601_20190601_plts/bias.svg
/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_NH_20180601_20190601_plts/stddev.svg


/data2/validations/validate_conc_ice_conc_polstere-100_amsr2_NH_20180601_20190601_plts/match.svg
/data2/validations/validate_conc_ice_conc_polstere-100_amsr2-tud_NH_20180601_20190601.nc


...

/data2/validations/validate_conc_ice_conc_polstere-100_multi_NH_20000101_20190630_plts/match.svg
/data2/validations/validate_conc_ice_conc_polstere-100_multi_SH_20000101_20190630.nc


/data2/validations/validate_conc_ice_conc_polstere-100_multi_SH_20000101_20190630_plts/bias.svg
/data2/validations/validate_conc_ice_conc_polstere-100_multi_SH_20000101_20190630_plts/stddev.svg


/data2/validations/validate_conc_ice_conc_polstere-100_multi_SH_20000101_20190630_plts/match.svg
/data2/validations/validate_conc_ice_conc_polstere-100_multi_NH_20000101_20190630.nc


/data2/validations/validate_conc_ice_conc_polstere-100_multi_NH_20000101_20190630_plts/bias.svg
/data2/validations/validate_conc_ice_conc_polstere-100_multi_NH_20000101_20190630_plts/stddev.svg


/data2/validations/validate_conc_ice_conc_polstere-100_multi_NH_20000101_20190630_plts/match.svg


## Plot Edge

In [12]:
def bar_plot(ds):
    p = figure(x_axis_type="datetime", plot_width=500, plot_height=int(500/1.618))
    p.output_backend = "svg"
    #p.title.text = 'Click on legend entries to mute the corresponding lines'
    width = 6
    df = ds[['true_ice', 'false_water', 'false_ice']].to_dataframe()
    total = (df.true_ice + df.false_water + df.false_ice) / 100
    df['true_ice'] /= total
    df['false_water'] /= total
    df['false_ice'] /= total
    df['1'], df['2'], df['3'] = df['true_ice'], df['false_water'], df['false_ice']
    df['2'] += df['1']
    df['3'] += df['2']
    width = 6 if len(df) < 100 else 0.5
    source = ColumnDataSource(df)
    p.vbar(x='time', source=source,
           bottom=0, top='1', width=width, line_width=width, color='grey', legend='True Ice')
    p.vbar(x='time', source=source,
           bottom='1', top='2', width=width, line_width=width, color='royalblue', legend='False Water')
    p.vbar(x='time', source=source,
           bottom='2', top='3', width=width, line_width=width, color='crimson', legend='False Ice')
    p.legend.location = "bottom_left"
    p.legend.label_text_font_size = '8pt'
    p.yaxis.axis_label = "Pixels (%)"
    return p
    
    
def make_plots(glob_str, from_date):
    for fn in glob(glob_str):
        ds = xr.open_mfdataset(fn)
        ds.attrs = {'filename': fn}
        p = bar_plot(ds)
        show(p)
        export_svgs(p, filename=save_fname(ds, 'match'))    
           

make_plots('/data2/validations/validate_edge_ice_edge_*.nc', '2000-01-01')            
# make_plots('/data2/validations/validate_edge_ice_edge_*.nc', '2018-06-01')

/data2/validations/validate_edge_ice_edge_polstere-100_multi_NH_20000101_20190601_plts/match.svg


/data2/validations/validate_edge_ice_edge_polstere-100_multi_SH_20180601_20190601_plts/match.svg


/data2/validations/validate_edge_ice_edge_polstere-100_multi_NH_20180601_20190601_plts/match.svg


/data2/validations/validate_edge_ice_edge_polstere-100_multi_SH_20000101_20190601_plts/match.svg
